# I/O: FITS y ASCII

En esta sección vamos a:

- Aprender a utilizar archivos FITS
    - <a href="#Abrir-un-archivo-FITS">Abrir un archivo FITS</a>
    - <a href="#Examinar-la-cabecera">Examinar la cabecera</a>
    - <a href="#Trabajando-con-imagenes-FITS">Trabajando con imagenes FITS</a>
    - <a href="#Trabajando-con-tablas-FITS">Trabajando con tablas FITS</a>
    - Visualizar con `matplotlib` y `astropy.visualization`
- Aprender a utilizar archivos ASCII
    - <a href="#Abrir-un-archivo-ASCII">Abrir un archivo ASCII</a>
    - Visualizar con `matplotlib` y `astropy.visualization`
    
**Para más información, consulte la documentación de Astropy sobre [FITS](https://docs.astropy.org/en/stable/io/fits/) y [ASCII](https://docs.astropy.org/en/stable/io/ascii/index.html).**

---

# Abrir un archivo FITS

In [ ]:
from astropy.io import fits

La función `open()` en [astropy.io.fits](https://docs.astropy.org/en/stable/io/fits/index.html) funciona tanto con archivos regulares como con archivos comprimidos.

Abra un archivo FITS y examine su contenido:

In [ ]:
with fits.open('j94f05bgq_flt.fits.gz') as f:
    f.info()

Abrir un archivo FITS utilizando la sentencia *with* de Python garantiza que el archivo se cierre sin necesidad de hacerlo explícitamente.

También existe un atajo para inspeccionar el contenido de un archivo FITS antes de abrirlo completamente:

In [ ]:
fits.info('j94f05bgq_flt.fits.gz')

In [ ]:
type(f)

Los objetos de archivo FITS, llamados `HDUList`, son similares a listas. Los HDU individuales se indexan como listas:

In [ ]:
f[1]

O utilizando una tupla del tipo (EXTNAME, EXTVER).

In [ ]:
f[("SCI", 1)]

## Examinar la cabecera

Los encabezados (headers) son similares a diccionarios de Python. Aquí vamos a examinar un encabezado, modificar una palabra clave existente y agregar un nuevo registro (card).

In [ ]:
keyname = 'CRPIX1'
extnum = 1

with fits.open('j94f05bgq_flt.fits.gz', mode='update') as f:
    value = f[extnum].header.get(keyname)  # None si no existe
    print("{}: {}".format(keyname, value))
    f[extnum].header[keyname] = value + 1  # Reasignar la palabra clave
    print("Actualizado {}: {}".format(keyname, f[extnum].header[keyname]))

Existen varias formas de agregar un nuevo registro (card) al encabezado.

Nota: Intentar acceder a una palabra clave que no existe genera un *KeyError*.

In [ ]:
f[extnum].header['observer'] = "Edwin Hubble"
f[extnum].header  # Desplázate hasta el final para ver OBSERVER

In [ ]:
# Usa "insert()" para controlar en qué lugar insertar la nueva palabra clave.
# Aquí, la tupla es (KEYWORD, VALUE, COMMENT).
f[extnum].header.insert('OBSERVER', ('NOBS', 2, "Numero de noches de observacion"))
f[extnum].header  # Desplázate hasta el final para ver NOBS insertado antes de OBSERVER

In [ ]:
# La palabra clave en el encabezado FITS tampoco distingue entre mayúsculas y minúsculas.
print('observer:', f[extnum].header['OBSERVER'])
print('{}: {}'.format(f[extnum].header.comments['NOBS'], f[extnum].header['NOBS']))

## Trabajando con imagenes FITS

In [ ]:
with fits.open('pix.fits.gz') as f:
    f.info()
    scidata = f[0].data.copy()

Una imagen es un NumPy array guardado como la parte de datos (data) de un HDU.

In [ ]:
print(scidata.shape)
print(scidata.dtype)

`scidata` es una copia del array de datos del HDU. Si lo modificas, no afectará los datos en el archivo FITS.

In [ ]:
# Todas las operaciones disponibles para NDArray se pueden aplicar al arreglo de datos FITS.
scidata[2:10, 3:7].mean()

`astropy.io.fits` ofrece funciones prácticas para trabajar con archivos FITS. Son útiles para consultas rápidas e interactivas. 

### Visualizando con `matplotlib`

In [ ]:
from astropy.visualization import imshow_norm, PercentileInterval, LogStretch
from matplotlib import pyplot as plt

%matplotlib inline

Para imágenes astronómicas, puedes usar `astropy.visualization` para [normalizar y estirar](https://docs.astropy.org/en/stable/visualization/normalization.html) la visualización. (`astropy.visualization` se cubrirá con más detalle en una sección posterior.) Aquí, queremos que sea logarítmica.

In [ ]:
# Mostrar la imagen (puedes ignorar las advertencias si aparecen)
fig, ax = plt.subplots()
im, norm = imshow_norm(
    scidata, ax, origin='lower',
        interval=PercentileInterval(99.9), stretch=LogStretch())
fig.colorbar(im)

`matplotlib` tiene varios mapas de color incorporados que se pueden usar para representar datos. Puedes ver todas las opciones en el [sitio web de matplotlib](https://matplotlib.org/examples/color/colormaps_reference.html).

In [ ]:
fig, ax = plt.subplots()
im, norm = imshow_norm(
    scidata, ax, origin='lower', cmap='gray',
    interval=PercentileInterval(99.9), stretch=LogStretch())
fig.colorbar(im)

## Trabajando con tablas FITS

**Nota**: El método recomendado para leer y escribir una sola tabla FITS es usar la [interfaz unificada de entrada/salida (I/O)](https://docs.astropy.org/en/stable/io/unified.html#table-io-fits):

    from astropy.table import Table
    t = Table.read('data.fits')

También mostramos un ejemplo usando [astropy.io.fits](https://docs.astropy.org/en/stable/io/fits/index.html#) a continuación, ya que existe mucho código heredado que lo utiliza:

    with fits.open('data.fits') as hdu_list:
        hdu_list.info()
        table_data = hdu_list[1].data
        print('Nombres de las columnas: \n', table_data.names)
        print('\nFila 1: \n', table_data[1])
        print('\nColumna "time": \n', table_data.field('time'))
        print('\nNúmero de filas: \n', len(table_data))

In [ ]:
from astropy.table import QTable
from astropy.utils.data import download_file

In [ ]:
table_filename = download_file('http://data.astropy.org/tutorials/FITS-tables/chandra_events.fits', cache=True)

In [ ]:
# Cuando una tabla contiene columnas con unidades, se puede usar QTable en lugar de Table.
tab = QTable.read(table_filename, hdu=1)

In [ ]:
tab.colnames

In [ ]:
tab[0]  # Primera fila

In [ ]:
tab['time']

In [ ]:
print("Numero de filas:", len(tab))

### Visualizando con `matplotlib`
#### Histograma 1D

In [ ]:
fig, ax = plt.subplots()
ax.hist(tab['energy'].value, bins=128, log=True)
ax.semilogy()
ax.set_xlabel("eV")

# El punto y coma al final suprime la salida adicional de matplotlib en notebooks.
ax.set_ylabel("Conteos por bin");

#### Histograma 2D

In [ ]:
from matplotlib.colors import LogNorm

fig, ax = plt.subplots()
h = ax.hist2d(tab['x'].value, tab['y'].value,
              bins=(1000, 1000), norm=LogNorm())
fig.colorbar(h[3]);

### Trabajando con archivos grandes

La función `open()` admite la opción `memmap=True`, que permite acceder a los datos del arreglo de cada HDU mediante mapeo de memoria, en lugar de cargarlos completamente en memoria de una sola vez. Esto es especialmente útil para trabajar con arreglos muy grandes que no caben por completo en la memoria física.

Para más detalles, consulta https://astropy.readthedocs.io/en/stable/io/fits/index.html#working-with-large-files.

---
# Abrir un archivo ASCII

¿Qué pasa si tus datos están en un archivo de texto? Hemos incluido `throughput.txt` como ejemplo.

    Valores de rendimiento del detector
    ——
    Longitud_onda_micrones Rendimiento
    2.0004663860000003 1.7911368754631042e-12
    2.002074478 2.5344376991635788e-12
    2.00368257 3.3872207023255276e-12
    2.005290662 2.891853959468528e-12
    ...         ...


Python tiene un método incorporado para abrir archivos de texto como archivos ASCII, pero no es muy amigable. Veamos cómo funciona:

In [ ]:
with open('throughput.txt') as f:
    throughput_data = f.read()

print(type(throughput_data))
print(len(throughput_data))

# ¿Y ahora qué? ¿Cómo extraemos los datos?

Afortunadamente, Astropy incluye un paquete para manejar archivos ASCII que separa inmediatamente los datos en filas y columnas, y los carga en una tabla fácil de usar:

In [ ]:
from astropy import units as u
from astropy.table import QTable

Como nuestro archivo tiene un par de líneas de comentarios al inicio, necesitamos indicarle al lector que empiece a leer el encabezado en la línea 2 y los datos en la línea 3. También renombraremos las columnas pasando `names`.

In [ ]:
throughput_data = QTable.read(
    'throughput.txt', format='ascii.basic', data_start=3, header_start=2,
    names=['lambda', 'rendimiento'])

# Asignar la unidad correcta a la columna de longitud de onda.
throughput_data['lambda'].unit = u.micron

throughput_data

El módulo ASCII de Astropy puede leer archivos con una variedad de formatos y extensiones, incluyendo separados por comas (CSV), delimitados por tabuladores, de ancho fijo, HTML, reStructuredText, y más. Consulta la [documentación de Astropy](https://docs.astropy.org/en/stable/io/ascii/#supported-formats) para una lista completa.

### Visualizando con `matplotlib`

In [ ]:
from astropy.visualization import quantity_support

with quantity_support():
    fig, ax = plt.subplots()
    ax.plot(throughput_data['lambda'], throughput_data['rendimiento'])
    ax.set_title('Ejemplo de Filtro')